In [78]:
# Adapted from sample digits recognition client on Scikit-Learn site.

import numpy as np
from sklearn import linear_model, datasets
from sklearn.cross_validation import train_test_split
from sklearn.neural_network import BernoulliRBM
from sklearn.pipeline import Pipeline
from sklearn.lda import LDA
from sklearn import linear_model, datasets, metrics


In [79]:
# import some data to play with
iris = datasets.load_iris()
iris.viewkeys()

dict_keys(['target_names', 'data', 'target', 'DESCR', 'feature_names'])

In [90]:
iris

{'DESCR': 'Iris Plants Database\n\nNotes\n-----\nData Set Characteristics:\n    :Number of Instances: 150 (50 in each of three classes)\n    :Number of Attributes: 4 numeric, predictive attributes and the class\n    :Attribute Information:\n        - sepal length in cm\n        - sepal width in cm\n        - petal length in cm\n        - petal width in cm\n        - class:\n                - Iris-Setosa\n                - Iris-Versicolour\n                - Iris-Virginica\n    :Summary Statistics:\n\n    ============== ==== ==== ======= ===== ====================\n                    Min  Max   Mean    SD   Class Correlation\n    ============== ==== ==== ======= ===== ====================\n    sepal length:   4.3  7.9   5.84   0.83    0.7826\n    sepal width:    2.0  4.4   3.05   0.43   -0.4194\n    petal length:   1.0  6.9   3.76   1.76    0.9490  (high!)\n    petal width:    0.1  2.5   1.20  0.76     0.9565  (high!)\n    ============== ==== ==== ======= ===== ====================\n\n

In [80]:
X = iris.data
Y = iris.target

In [81]:
X = (X - np.min(X, 0)) / (np.max(X, 0) + 0.0001)  # 0-1 scaling

In [82]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, 
                                test_size=0.2, random_state=10)

In [83]:
# Models we will use
logistic = linear_model.LogisticRegression()
rbm = BernoulliRBM(random_state=0, verbose=True)

classifier = Pipeline(steps=[('rbm', rbm), ('logistic', logistic)])

In [84]:
lda = LDA(n_components=3)

In [85]:
rbm.learning_rate = 0.06
rbm.n_iter = 10

# More components tend to give better prediction performance, but larger
# fitting time
rbm.n_components = 100
logistic.C = 6000.0

# Training RBM-Logistic Pipeline
classifier.fit(X_train, Y_train)

[BernoulliRBM] Iteration 1, pseudo-likelihood = -2.19, time = 0.00s
[BernoulliRBM] Iteration 2, pseudo-likelihood = -2.21, time = 0.00s
[BernoulliRBM] Iteration 3, pseudo-likelihood = -2.12, time = 0.00s
[BernoulliRBM] Iteration 4, pseudo-likelihood = -2.12, time = 0.00s
[BernoulliRBM] Iteration 5, pseudo-likelihood = -2.30, time = 0.00s
[BernoulliRBM] Iteration 6, pseudo-likelihood = -2.04, time = 0.00s
[BernoulliRBM] Iteration 7, pseudo-likelihood = -2.22, time = 0.00s
[BernoulliRBM] Iteration 8, pseudo-likelihood = -2.13, time = 0.00s
[BernoulliRBM] Iteration 9, pseudo-likelihood = -2.18, time = 0.00s
[BernoulliRBM] Iteration 10, pseudo-likelihood = -2.26, time = 0.00s


Pipeline(steps=[('rbm', BernoulliRBM(batch_size=10, learning_rate=0.06, n_components=100, n_iter=10,
       random_state=0, verbose=True)), ('logistic', LogisticRegression(C=6000.0, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False))])

In [86]:
# Training Logistic regression
logistic_classifier = linear_model.LogisticRegression(C=100.0)
logistic_classifier.fit(X_train, Y_train)

LogisticRegression(C=100.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [87]:
# Get predictions
print "The Logistic model using RBM features:"
print "Predict: ", classifier.predict(X_test)
print "Real:    ", Y_test

print

print "The Logistic model using raw features:"
print "Predict: ", logistic_classifier.predict(X_test)
print "Real:    ", Y_test

print

print "Linear Discriminant Analysis: "
lda.fit(X_train, Y_train)
print "Predict: ", lda.predict(X_test)
print "Real:    ", Y_test  

The Logistic model using RBM features:
Predict:  [1 2 0 2 0 1 2 1 0 1 2 2 1 0 0 2 2 0 0 0 2 2 2 0 2 0 1 1 2 2]
Real:     [1 2 0 1 0 1 1 1 0 1 1 2 1 0 0 2 1 0 0 0 2 2 2 0 1 0 1 1 1 2]

The Logistic model using raw features:
Predict:  [1 2 0 1 0 1 1 1 0 1 1 2 1 0 0 2 2 0 0 0 2 2 2 0 1 0 1 1 1 2]
Real:     [1 2 0 1 0 1 1 1 0 1 1 2 1 0 0 2 1 0 0 0 2 2 2 0 1 0 1 1 1 2]

Linear Discriminant Analysis: 
Predict:  [1 2 0 1 0 1 1 1 0 1 1 2 1 0 0 2 1 0 0 0 2 2 2 0 1 0 1 1 1 2]
Real:     [1 2 0 1 0 1 1 1 0 1 1 2 1 0 0 2 1 0 0 0 2 2 2 0 1 0 1 1 1 2]


In [88]:
###############################################################################
# Evaluation

print
print("Logistic regression using RBM features:\n%s\n" % (
    metrics.classification_report(
        Y_test,
        classifier.predict(X_test))))
print("Logistic regression using raw pixel features:\n%s\n" % (
    metrics.classification_report(
        Y_test,
        logistic_classifier.predict(X_test))))


Logistic regression using RBM features:
             precision    recall  f1-score   support

          0       1.00      1.00      1.00        10
          1       1.00      0.54      0.70        13
          2       0.54      1.00      0.70         7

avg / total       0.89      0.80      0.80        30


Logistic regression using raw pixel features:
             precision    recall  f1-score   support

          0       1.00      1.00      1.00        10
          1       1.00      0.92      0.96        13
          2       0.88      1.00      0.93         7

avg / total       0.97      0.97      0.97        30


